# EDA
#### Análisis Exploratorio de los Datos para la elección un Modelo de Clasificación de un Sistema de Recomendación de Peliculas por Similaridad

### Indice:
1. Introducción
2. Objetivos
3. Estadística Descriptiva
4. Tratamiento de Outliers
5. Agrupación de datos
6. Control de valores NAN
7. Análisis de correlación Lineal y Polinómico
8. Similaridad por puntuación de películas
8. Nubes de Palabras para los títulos de peliculas
9. Determinación de Variables reelevantes
10. Determinación del Modelo factible
12. Similitud del coseno
11. Conclusiones

### 1. Introducción
##### El presente notebook trata acerca del análisis estadístico de los datos que nos permitirá sacar inferencias e hipotesis para un modelo de clasificación 
##### Se trabajará con datos de Features de películas, en el cual se extraerán patrones y hallazgos para tener una mayor comprensión de los mismos.

### 2. Objetivos
##### 2.1 Determinar patrones y variables reelevantes en los datos
##### 2.2 Determinar los posibles modelos de clasificación 
##### 2.3 Sustentar las variables involucradas para el modelo de clasificación más factible

## 1.  Estadística Descriptiva

In [36]:
# Se importan las bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats 

In [55]:
# Se ingesta el dataset preprocesado
df = pd.read_csv('data_set/ds_clean.csv')

##### Se filtrarán las columnas con los siguientes criterios: #
- Las variables tienen que aportar información acerca de la naturaleza del contenido de cada película
- Las variables tienen que explicar con cierto grado la similaridad de contenido entre las películas

In [56]:
# Se filtran columnas innecesarias
df = df.drop('id', axis = 1)

In [57]:
# Reducción de dimencionalidad
df = df.drop('revenue', axis = 1)
df = df.drop('release_year', axis = 1)
df = df.drop('release_date', axis = 1)
df = df.drop('budget', axis = 1)
df = df.drop('original_language', axis = 1)
df = df.drop('runtime', axis = 1)
df = df.drop('production_countries', axis = 1)
df = df.drop('spoken_languages', axis = 1)
df = df.drop('status', axis =1)
df = df.drop('return', axis =1)

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45451 entries, 0 to 45450
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4497 non-null   object 
 1   genres                 43067 non-null  object 
 2   overview               44510 non-null  object 
 3   popularity             45451 non-null  float64
 4   production_companies   33634 non-null  object 
 5   tagline                20425 non-null  object 
 6   title                  45451 non-null  object 
 7   vote_average           45451 non-null  float64
 8   vote_count             45451 non-null  float64
 9   cast                   43097 non-null  object 
 10  crew                   44728 non-null  object 
 11  director               44728 non-null  object 
dtypes: float64(3), object(9)
memory usage: 4.2+ MB


In [59]:
# Medidas de Tendencia central de variables numéricas
df.describe(include=[np.number])

,popularity,vote_average,vote_count
count,45451.000000,45451.000000,45451.000000
mean,2.926887,5.624085,109.989703
std,6.006458,1.915424,491.352336
min,0.000000,0.000000,0.000000
25%,0.389095,5.000000,3.000000
50%,1.131386,6.000000,10.000000
75%,3.694370,6.800000,34.000000
max,547.488298,10.000000,14075.000000


In [60]:
# Caraterísticas de las variables numéricas categóricas
df.describe(include=object)

,belongs_to_collection,genres,overview,production_companies,tagline,title,cast,crew,director
count,4497,43067,44510,33634,20425,45451,43097,44728,44728
unique,1695,4064,44231,22666,20269,42195,42656,42908,20603
top,The Bowery Boys,Drama,No overview found.,Metro-Goldwyn-Mayer (MGM),Which one is the first to return - memory or t...,Blackout,Georges Méliès,Georges Méliès,Avy Kaufman
freq,29,5006,133,742,9,13,24,35,124


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45451 entries, 0 to 45450
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4497 non-null   object 
 1   genres                 43067 non-null  object 
 2   popularity             45451 non-null  float64
 3   production_companies   33634 non-null  object 
 4   tagline                20425 non-null  object 
 5   title                  45451 non-null  object 
 6   vote_average           45451 non-null  float64
 7   vote_count             45451 non-null  float64
 8   cast                   43097 non-null  object 
 9   crew                   44728 non-null  object 
 10  director               44728 non-null  object 
dtypes: float64(3), object(8)
memory usage: 3.8+ MB


In [33]:
df['vote_average']

0        7.7
1        6.9
2        6.5
3        6.1
4        5.7
        ... 
45446    5.7
45447    9.0
45448    3.8
45449    0.0
45450    0.0
Name: vote_average, Length: 45451, dtype: float64

In [34]:
df['vote_count']

0        5415.0
1        2413.0
2          92.0
3          34.0
4         173.0
          ...  
45446      26.0
45447       3.0
45448       6.0
45449       0.0
45450       0.0
Name: vote_count, Length: 45451, dtype: float64